In [50]:
import scipy.io as sio
import pandas as pd
import numpy as np
import itertools

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [98]:
DIM_SIZE = 200
NUM_NEG_SAMPLES = 10

In [5]:
lex = sio.loadmat('data/gold_standard.mat')['gold_standard']['map']
world = sio.loadmat('data/world.mat')['world']
corpus = sio.loadmat('data/corpus.mat')['corpus']

# World

In [113]:
def get_world():
    
    world = sio.loadmat('data/world.mat')['world']
    world = world[0][0]
    
    word_idx = world[0][0] - 1
    words = list(itertools.chain.from_iterable(map(list, world[1][0])))  
    word_dict = dict(zip(word_idx, words))
    
    obj_idx = world[3][0] - 1
    objs = list(itertools.chain.from_iterable(map(list, world[4][0])))  
    obj_dict = dict(zip(obj_idx, objs))
    
    return word_dict, obj_dict

# Gold Standard Lexicon

In [127]:
def get_gold_lexicon(word_dict, obj_dict):
    
    lex = sio.loadmat('data/gold_standard.mat')['gold_standard']['map']
    lex = lex[0][0]
    
    lex_dict_index = {}
    lex_dict_readable = {}
    for i, j in zip(lex[0], lex[1]):
        lex_dict_index[i-1] = j-1
        lex_dict_readable[word_dict[i-1]] = obj_dict[j-1]
        
    return lex_dict_index, lex_dict_readable

In [128]:
word_dict, obj_dict = get_world()
get_gold_lexicon(word_dict, obj_dict)

{0: 'a', 1: 'abc', 2: 'about', 3: 'again', 4: 'ah', 5: 'ahhah', 6: 'alison', 7: 'alisons', 8: 'all', 9: 'almost', 10: 'alphabet', 11: 'always', 12: 'ambrose', 13: 'and', 14: 'animals', 15: 'another', 16: 'any', 17: 'anymore', 18: 'are', 19: 'arent', 20: 'around', 21: 'as', 22: 'at', 23: 'aw', 24: 'b', 25: 'baa', 26: 'baby', 27: 'back', 28: 'backs', 29: 'ball', 30: 'balls', 31: 'baseball', 32: 'bathroom', 33: 'be', 34: 'beads', 35: 'beak', 36: 'bear', 37: 'because', 38: 'belongs', 39: 'better', 40: 'big', 41: 'bigbird', 42: 'bigbirds', 43: 'bird', 44: 'birdie', 45: 'bit', 46: 'blue', 47: 'boo', 48: 'book', 49: 'books', 50: 'bottle', 51: 'bottles', 52: 'bowtie', 53: 'boy', 54: 'bracelet', 55: 'bubbles', 56: 'bumble', 57: 'bumblebee', 58: 'bumpba', 59: 'bunnies', 60: 'bunny', 61: 'bunnyrabbit', 62: 'by', 63: 'bzz', 64: 'c', 65: 'can', 66: 'cant', 67: 'carpet', 68: 'chair', 69: 'chew', 70: 'choo', 71: 'color', 72: 'colors', 73: 'come', 74: 'comes', 75: 'could', 76: 'courtney', 77: 'cow', 7

({26: 0,
  36: 1,
  41: 2,
  42: 2,
  43: 7,
  48: 3,
  49: 3,
  60: 5,
  61: 5,
  77: 6,
  78: 6,
  233: 6,
  234: 6,
  102: 7,
  103: 7,
  110: 8,
  142: 11,
  146: 12,
  183: 13,
  184: 13,
  185: 13,
  191: 14,
  192: 14,
  226: 16,
  277: 17,
  278: 17,
  279: 17,
  290: 18,
  296: 19,
  297: 19,
  308: 20,
  59: 5,
  44: 7},
 {'baby': 'baby',
  'bear': 'bear',
  'bigbird': 'bird',
  'bigbirds': 'bird',
  'bird': 'duck',
  'book': 'book',
  'books': 'book',
  'bunny': 'bunny',
  'bunnyrabbit': 'bunny',
  'cow': 'cow',
  'cows': 'cow',
  'moocow': 'cow',
  'moocows': 'cow',
  'duck': 'duck',
  'duckie': 'duck',
  'eyes': 'eyes',
  'hand': 'hand',
  'hat': 'hat',
  'kitty': 'kitty',
  'kittycat': 'kitty',
  'kittycats': 'kitty',
  'lamb': 'lamb',
  'lambie': 'lamb',
  'mirror': 'mirror',
  'pig': 'pig',
  'piggie': 'pig',
  'piggies': 'pig',
  'rattle': 'rattle',
  'ring': 'ring',
  'rings': 'ring',
  'sheep': 'sheep',
  'bunnies': 'bunny',
  'birdie': 'duck'})

In [116]:
gold_idx

{27: 1,
 37: 2,
 42: 3,
 43: 3,
 44: 8,
 49: 4,
 50: 4,
 61: 6,
 62: 6,
 78: 7,
 79: 7,
 234: 7,
 235: 7,
 103: 8,
 104: 8,
 111: 9,
 143: 12,
 147: 13,
 184: 14,
 185: 14,
 186: 14,
 192: 15,
 193: 15,
 227: 17,
 278: 18,
 279: 18,
 280: 18,
 291: 19,
 297: 20,
 298: 20,
 309: 21,
 60: 6,
 45: 8}

# Corpus


In [135]:
def get_corpus(word_dict, obj_dict):
    
    corpus = sio.loadmat('data/corpus.mat')['corpus']
    corpus = corpus[0]
    
    corpus_idx = []
    corpus_readable = []
    for pair in corpus:
        
        scene = list(pair[0][0]-1)
        utt = list(pair[1][0]-1)
        new_pair = {'scene': scene, 'utt': utt}
        corpus_idx.append(new_pair)
        
        scene_word = list(map(lambda x: obj_dict[x], scene))
        utt_word = list(map(lambda x: word_dict[x], utt))
        new_pair_readable = {'scene': scene_word, 'utt': utt_word}
        corpus_readable.append(new_pair_readable)
    
    return corpus_idx, corpus_readable

{'scene': [3, 2, 18, 9], 'utt': [5, 204, 390, 65, 292, 49, 83]}
{'scene': ['book', 'bird', 'rattle', 'face'], 'utt': ['ahhah', 'look', 'we', 'can', 'read', 'books', 'david']}


# Helper Functions

In [18]:
def get_alignments(pair):
    utt, scene = pair['utt'], pair['scene']
    combos = list(itertools.product(utt, scene))
    return combos

In [70]:
def create_embeddings(vocab_len, obj_len):
    # Access with torch.LongTensor()
    # word_embeddings(torch.LongTensor([0, 1, 2])).shape = (3, 200)
    word_embeddings = EmbeddingModeler(vocab_len, DIM_SIZE)
    obj_embeddings = EmbeddingModeler(obj_len, DIM_SIZE)
    return word_embeddings, obj_embeddings

# Neural Network Functions

In [69]:
class EmbeddingModeler(nn.Module):

    def __init__(self, vocab_len, embedding_size):
        super(EmbeddingModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_len, embedding_size)
    
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        return embeds

In [80]:
def apl_loss(word_embeddings, object_embeddings, obj_len, combo, scene):
    word, obj = combo[0] - 1, combo[1] - 1 # 1 indexing
    neg_samples = np.random.choice(np.setdiff1d(np.arange(obj_len) + 1, scene), NUM_NEG_SAMPLES) - 1 #1 indexing 
    word_embedding = word_embeddings(word)
    torch.sum(torch.max())
    
    return torch.max() #TODO: make this vectorized, so i=nstead of max ocompared to 0, has to be max compares to torch.zeros)

def asl_loss(word_embeddings, object_embeddings, vocab_len):
    pass

def joint_loss(word_embeddings, object_embeddings, obj_len, vocab_len):
    pol_loss = apl_loss()
    syn_loss = asl_loss()
    return torch.add(pol_loss, syn_loss)

In [97]:
np.setdiff1d(np.arange(obj_len), [4, 9, 3])

array([ 0,  1,  2,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21])

In [93]:
np.random.choice(np.arange(10), 10, replace=False)

array([3, 8, 2, 4, 6, 1, 7, 0, 9, 5])

In [83]:
def train_embeddings(corpus_idx, corpus_readable, vocab_len, obj_len, loss_type):
    
    word_embeddings, obj_embeddings = create_embeddings(vocab_len, obj_len)
    losses = []
    for epoch in range(10):
        total_loss = 0
        for i, pair in enumerate(corpus_idx):
            
            readable = corpus_readable[i]
            print(f"UTT: {pair['utt']}")
            print(f"SCENE: {pair['scene']}")
            print(f"UTT: {readable['utt']}")
            print(f"SCENE: {readable['scene']}")
            
            combos = get_alignments(pair)
            for combo in combos:
                
                if loss_type == 'anti-polysemy':
                    loss = apl_loss(word_embeddings, obj_embeddings, obj_len, combo, pair['scene'])
                elif loss_type == 'anti-synonymy':
                    loss = asl_loss(word_embeddings, obj_embeddings, vocab_len, combo)
                else:
                    loss = joint_loss(word_embeddings, obj_embeddings, obj_len, vocab_len, combo)

            loss.backward()
    
    return losses
    
    

In [84]:
train_embeddings(corpus_idx, corpus_readable, vocab_len, obj_len, 'anti-polysemy')

UTT: [6, 205, 391, 66, 293, 50, 84]
SCENE: [4, 3, 19, 10]
UTT: ['ahhah', 'look', 'we', 'can', 'read', 'books', 'david']
SCENE: ['book', 'bird', 'rattle', 'face']
(6, 4)


TypeError: max() received an invalid combination of arguments - got (), but expected one of:
 * (Tensor input)
 * (Tensor input, Tensor other, Tensor out)
 * (Tensor input, int dim, bool keepdim, tuple of Tensors out)


In [25]:
word_dict, obj_dict = get_world()
gold_idx, gold_readable = get_gold_lexicon(word_dict, obj_dict)
corpus_idx, corpus_readable = get_corpus(word_dict, obj_dict)

vocab_len = len(word_dict)
obj_len = len(obj_dict)

# Framework

In [66]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
#         print(self.embeddings.weight.shape)
        embeds = self.embeddings(inputs).view((1, -1))
#         print(embeds.shape)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)


[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [68]:
for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
#         print(context_idxs)
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
#         print(log_probs.shape)
        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The los

[515.5346808433533, 513.2982707023621, 511.07346773147583, 508.8592507839203, 506.6559977531433, 504.46337628364563, 502.276953458786, 500.094162940979, 497.91767406463623, 495.74654150009155, 493.5804042816162]
